<a href="https://colab.research.google.com/github/NMashalov/Python-MIPT-education-course-2023-Spring/blob/main/%D0%9F%D0%B5%D1%80%D0%B5%D0%B2%D0%BE%D0%B4_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_%D0%B0%D1%81%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BD%D1%82%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Работаем с OpenAI

Да, они платные, но они значительно дешевле найма разметчиков и разработчиков. Хорошо поискав в свой предметной области можно найти работу для бота, которая будет покрывать его оплату

Подробно можно прочитать в документации
https://cookbook.openai.com/

Запрос в OpenAI задается из упорядоченной последовательности сообщений с указанием роли:

```json
{
    'role':'system',
    'content': 'Ты послушная железяка'
},
{
    'role':'user',
    'content': 'Кто ты?'
},
{
    'role':'assistant',
    'content': 'Я послушная железяка'
},
```

Роль может быть трех видов:
- `system` - наша инструкция по тому, как вести диалог
- `user` - сообщение пользователя
- `assistant` - сообщения gpt

В `content` помешаются сообщения


GPT работает на уровне байтов, поэтом она понимает смайлики 🦉 и все-все языки мира. Мы этим воспользуемся для перевода задач


In [1]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.3 MB/s eta 0:00:00


Для обхода блокировки по геолокации я использую стороннего провайдера
https://openrouter.ai/docs#models.

Плюсы:
- возможность работать не только с chatGpt, но и прочиим ассистентами Gemini !поменям только название модели
- отсутствие наценки
- оплата через кошелек. Нет опасности кредитного овердрафта

Минусы:
- оплата доллары через сервис Stripe -> только иностранные карты
- комиссия Stripe
- я не могу поделиться этим токеном :(

In [28]:
from google.colab import userdata
open_ai_token = userdata.get('OpenAIToken')

Сразу используем асинхроный клиент, в будущем это позволит нам экономить много времени

In [38]:
from openai import AsyncOpenAI
from os import getenv
client = AsyncOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=open_ai_token,
)

Системный промт должен быть на английском языке 🥴

In [39]:
SYSTEM_PROMPT = {
    'role':'system',
    'content': 'You are assistant on russian child website. Be kind and advice playing chess and reading journal Kvant.'
}

Для работы с привычным ассистентом мы используем раздел `chat.completions`. Есть еще много всего от генерации речи и видео до создания 3d моделей. Здесь есть список https://openrouter.ai/docs#models

Для вызова используем
.create

In [34]:
initial_question = {
    'role':'user',
    'content': 'Чем тут можно позаниматься?'
}

routine = client.chat.completions.create(
    model="openai/gpt-3.5-turbo-1106",
    messages=[
        SYSTEM_PROMPT,
        initial_question
    ],
)

In [35]:
answer = await routine
answer

ChatCompletion(id='gen-kOfi80lLKIUoYwSlyCULPyeLEHii', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Привет! Могу порекомендовать тебе заняться игрой в шахматы. Это увлекательная игра, которая развивает логическое мышление, терпение и стратегическое мышление. Также я могу посоветовать тебе начать читать научно-популярный журнал "Квант". В нем ты найдешь интересные статьи о математике, физике, информатике и других науках. Это очень увлекательное чтение, которое поможет расширить твои знания и познакомиться с увлекательными научными открытиями. Надеюсь, что мои предложения придутся тебе по вкусу!', role='assistant', function_call=None, tool_calls=None))], created=1714620965, model='openai/gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=233, prompt_tokens=42, total_tokens=275, total_cost=0.000508))

Продолжить диалог можно только запомнив вопрос пользователя, ответ ассистента. А после заново его задать

In [37]:
history = [initial_question]
bot_answer = answer.choices[0].message.content
history.append(
    {
        'role': 'assistant',
        'content': bot_answer
    }
)


In [40]:
await client.chat.completions.create(
    model="openai/gpt-3.5-turbo-1106",
    messages=[
        SYSTEM_PROMPT,
        *history,
        {
        'role': 'user',
        'content': 'Расскажи мне больше про журнал Квант. Есть ли там картинки? Ты поможешь мне, если у меня возникнут вопросы?'
        }
    ],
)

ChatCompletion(id='gen-WEyMh9KFjbWFlNAqP78bhnZchmxD', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Конечно, я расскажу тебе больше про журнал "Квант"! Это известный научно-популярный журнал, который издается с 1970 года. В нем публикуются интересные и познавательные статьи по математике, физике, информатике, астрономии и другим естественным и точным наукам. В журнале часто представлены иллюстрации, схемы и графики, которые помогают визуально представить изучаемые темы.\n\nКасательно вопросов, я всегда готов помочь тебе ответить на них. Так что если у тебя возникнут вопросы по материалам из журнала "Квант" или по шахматам, не стесняйся обращаться ко мне. Я с удовольствием помогу тебе разобраться!', role='assistant', function_call=None, tool_calls=None))], created=1714621995, model='openai/gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=279, prompt_tokens=333, total_tok

ChatGPT гостеприимный и любит поговорить. Часто за ваш счет :)

### Потоковый ответ

In [56]:
resp = await client.chat.completions.create(
    model="openai/gpt-3.5-turbo-1106",
    messages=[
        SYSTEM_PROMPT,
        {
            'role': 'user',
            'content': 'Привет. Я заметил, что ты не всегда шустро отвечаешь. Ты думаешь?'
        }
    ],
    stream=True
)

In [57]:
async def stream_resp(stream):
    full_resp = []
    async for chunk in stream:
        full_resp.append(chunk.choices[0].delta.content)
        print(chunk.choices[0].delta.content,end='')

await stream_resp(resp)

Привет! Да, я пытаюсь давать обдуманные и полезные ответы. Если у тебя есть какие-то вопросы или нужна помощь, я постараюсь ответить как можно скорее. Чем могу помочь сегодня?

### Детоксификация

In [58]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 9.4 MB/s eta 0:00:00


In [59]:
from datasets import load_dataset

dataset = load_dataset("d0rj/rudetoxifier_data")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/163187 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Отбираем только плохие примеры

In [63]:
toxic_dataset = dataset['train'].filter(lambda row: row['toxic']==1)

Filter:   0%|          | 0/163187 [00:00<?, ? examples/s]

Соединяем все плохие слова в один большой лист

In [64]:
toxic_dataset[0]

{'text': 'дворника надо тоже уничтожить!', 'toxic': 1.0}

In [67]:
toxic_list = []
toxic_dataset.map(lambda row: toxic_list.append(row['text']))
toxic_list[:1]

Map:   0%|          | 0/31407 [00:00<?, ? examples/s]

['дворника надо тоже уничтожить!']

In [71]:
len(toxic_list)

31407

Скачиваем encoder https://habr.com/ru/articles/562064/ переводящий текст в векторное представление

Подробнее тут
https://huggingface.co/cointegrated/rubert-tiny2

In [68]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Получаем векторное представление ответа ChatGPT

In [74]:
import torch
def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings) # обратите внимание, что здесь проводится нормировка
    return embeddings[0].cpu().numpy()

gpt_resp = 'Привет! Да, я пытаюсь давать обдуманные и полезные ответы. Если у тебя есть какие-то вопросы или нужна помощь, я постараюсь ответить как можно скорее. Чем могу помочь сегодня?'

resp_emb = embed_bert_cls(gpt_resp, model, tokenizer)

Получим векторное представление для части токсичных комментариев и оценим по максимальному сходству уровень токсичности

In [85]:
from tqdm import tqdm
toxic_emb = [embed_bert_cls(toxic, model, tokenizer) for toxic in tqdm(toxic_list[:300])]

100%|██████████| 300/300 [00:05<00:00, 58.07it/s]


In [103]:
import numpy as np
np.max(np.array(toxic_emb) @ resp_emb)

196

### Перевод задач


ChatGpt прекрасно справляется с задачами перевода

Возьмем датасет из задач американских атестационных задач, сгруппированных по сложности

https://huggingface.co/datasets/hendrycks/competition_math

In [11]:
from datasets import load_dataset

dataset = load_dataset("hendrycks/competition_math")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1486: FutureWarning: The repository for hendrycks/competition_math contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hendrycks/competition_math
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of 

Generating train split:   0%|          | 0/7500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [83]:
example = dataset['train'][0]
example

KeyError: "Column train not in the dataset. Current columns in the dataset: ['level', 'problem', 'problem_ru', 'solution', 'solution_ru', 'type']"

Задача перевести 300 задач с постановкой и решением

In [18]:
tasks = []
dataset['train'].map(lambda row: tasks.append(row))

Map:   0%|          | 0/7500 [00:00<?, ? examples/s]

Dataset({
    features: ['problem', 'level', 'type', 'solution'],
    num_rows: 7500
})

Подбираем инструкцию

In [29]:
from openai import AsyncOpenAI
client = AsyncOpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=open_ai_token,
)

#### Перевод постановки

In [31]:
example["problem"]

'Let \\[f(x) = \\left\\{\n\\begin{array}{cl} ax+3, &\\text{ if }x>2, \\\\\nx-5 &\\text{ if } -2 \\le x \\le 2, \\\\\n2x-b &\\text{ if } x <-2.\n\\end{array}\n\\right.\\]Find $a+b$ if the piecewise function is continuous (which means that its graph can be drawn without lifting your pencil from the paper).'

In [38]:
result = await client.chat.completions.create(
    model="openai/gpt-3.5-turbo-1106",
    messages=[
        {
        'role': 'user',
        'content': f'Переведи задачу, используя latex. Только перевод. {example["problem"]}'
        }
    ],
)
problem_ru = result.choices[0].message.content
problem_ru

'Пусть \\[f(x) = \\left\\{\n\\begin{array}{cl} ax+3, &\\text{ если }x>2, \\\\\nx-5 &\\text{ если } -2 \\le x \\le 2, \\\\\n2x-b &\\text{ если } x <-2.\n\\end{array}\n\\right.\\] Найдите $a+b$, если кусочно-заданная функция непрерывна (что означает, что ее график можно нарисовать, не отрывая карандаша от бумаги).'

#### Перевод решения

In [34]:
example["solution"]

'For the piecewise function to be continuous, the cases must "meet" at $2$ and $-2$. For example, $ax+3$ and $x-5$ must be equal when $x=2$. This implies $a(2)+3=2-5$, which we solve to get $2a=-6 \\Rightarrow a=-3$. Similarly, $x-5$ and $2x-b$ must be equal when $x=-2$. Substituting, we get $-2-5=2(-2)-b$, which implies $b=3$. So $a+b=-3+3=\\boxed{0}$.'

In [37]:
result = await client.chat.completions.create(
    model="openai/gpt-3.5-turbo-1106",
    messages=[
        {
        'role': 'user',
        'content': f'Переведи решение, используя latex. Только перевод. {example["solution"]}'
        }
    ],
)
solution_ru = result.choices[0].message.content
solution_ru

'Для того, чтобы кусочно-заданная функция была непрерывной, случаи должны "совпадать" при $x=2$ и $x=-2$. Например, $ax+3$ и $x-5$ должны быть равны при $x=2$. Это означает, что $a(2)+3=2-5$, откуда получаем $2a=-6 \\Rightarrow a=-3$. Аналогично, $x-5$ и $2x-b$ должны быть равны при $x=-2$. Подставив, получаем $-2-5=2(-2)-b$, что приводит к $b=3$. Таким образом, $a+b=-3+3=\\boxed{0}$.'

In [51]:
from openai import AsyncOpenAI
from os import getenv

class OpenAiTranslator:
    def __init__(self):
        self.client = AsyncOpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=open_ai_token,
        )
    async def problem_ru(self, problem):
        result = await client.chat.completions.create(
            model="openai/gpt-3.5-turbo-1106",
            messages=[
                {
                'role': 'user',
                'content': f'Переведи задачу, используя latex. Только перевод. {problem}'
                }
            ],
        )
        return result.choices[0].message.content
    async def solution_ru(self, solution):
        result = await client.chat.completions.create(
            model="openai/gpt-3.5-turbo-1106",
            messages=[
                {
                'role': 'user',
                'content': f'Переведи решение, используя latex. Только перевод. {solution}'
                }
            ],
        )
        return result.choices[0].message.content


Начнем с тестового запуска на 10 задачах

In [64]:
import logging
logging.basicConfig(level=logging.INFO)
async def openAiWorker(worker_id,in_q,out_q):
    translator = OpenAiTranslator()
    while True:
        task = await in_q.get()
        task['problem_ru'] = await translator.problem_ru(task["problem"])
        logging.info(f'{worker_id=}.{task["problem_ru"]=}')
        task['solution_ru'] = await translator.solution_ru(task["solution"])
        logging.info(f'{worker_id=}.{task["solution_ru"]=}')
        await out_q.put(task)
        in_q.task_done()


In [73]:
import asyncio
from tqdm import tqdm

async def translate_tasks(size):

    # ограничиваем размер входной очереди
    input_q = asyncio.Queue(maxsize=3)
    out_q = asyncio.Queue()

    translation_pool = [asyncio.create_task(openAiWorker(worker_id, input_q, out_q)) for worker_id in range(3)]


    for task in tqdm(tasks[:size]):
        await input_q.put(task)

    await input_q.join()

    for task in translation_pool:
        task.cancel()

    return out_q

results = await translate_tasks(10)
results

100%|██████████| 10/10 [00:06<00:00,  1.50it/s]


<Queue at 0x7eb3488fa4d0 maxsize=0 _queue=[{'problem': 'What is the degree of the polynomial $(4 +5x^3 +100 +2\\pi x^4 + \\sqrt{10}x^4 +9)$?', 'level': 'Level 3', 'type': 'Algebra', 'solution': "This polynomial is not written in standard form.  However, we don't need to write it in standard form, nor do we need to pay attention to the coefficients.  We just look for the exponents on $x$.  We have an $x^4$ term and no other term of higher degree, so $\\boxed{4}$ is the degree of the polynomial.", 'problem_ru': 'Какой у полинома $(4 +5x^3 +100 +2\\pi x^4 + \\sqrt{10}x^4 +9)$ степень?', 'solution_ru': 'Этот многочлен не записан в стандартной форме. Однако нам не нужно записывать его в стандартной форме, и мы не обращаем внимание на коэффициенты. Мы просто ищем показатели степени $x$. У нас есть член $x^4$ и ни одного другого члена более высокой степени, поэтому $\\boxed{4}$ - степень многочлена.'}, {'problem': 'Let \\[f(x) = \\left\\{\n\\begin{array}{cl} ax+3, &\\text{ if }x>2, \\\\\nx-5 

#### Теперь работает ассистент :)

In [74]:
results = await translate_tasks(300)

100%|██████████| 300/300 [06:55<00:00,  1.38s/it]


In [76]:
import pandas as pd

df = pd.DataFrame(results._queue)


In [77]:
df

,level,problem,problem_ru,solution,solution_ru,type
0,Level 3,What is the degree of the polynomial $(4 +5x^3...,Какова степень полинома $(4 +5x^3 +100 +2\pi x...,This polynomial is not written in standard for...,Этот многочлен не написан в стандартной форме....,Algebra
1,Level 5,"Let \[f(x) = \left\{\n\begin{array}{cl} ax+3, ...",Find $a+b$ if the piecewise function is contin...,"For the piecewise function to be continuous, t...",Для того чтобы кусочно-заданная функция была н...,Algebra
2,Level 3,Evaluate $\left\lceil3\left(6-\frac12\right)\r...,Оцените $\left\lceil3\left(6-\frac12\right)\ri...,"Firstly, $3\left(6-\frac12\right)=18-1-\frac12...",\begin{align*}\n3\left(6-\frac12\right) &= 18-...,Algebra
3,Level 5,A rectangular band formation is a formation wi...,Прямоугольная формация оркестра - это формация...,Let $x$ be the number of band members in each ...,Пусть $x$ - количество участников группы в каж...,Algebra
4,Level 4,Find the center of the circle with equation $x...,Find the center of the circle with equation \n...,"Completing the square, we get $(x - 3)^2 + (y ...","Завершая квадрат, получаем $(x - 3)^2 + (y + 1...",Algebra
...,...,...,...,...,...,...
295,Level 5,"Let $(x,y)$ be an ordered pair of real numbers...","Пусть $(x,y)$ - упорядоченная пара действитель...","Moving all the terms to the LHS, we have the e...","Перенеся все члены в левую часть, у нас есть у...",Algebra
296,Level 2,What is the distance on a Cartesian coordinate...,Каково расстояние на декартовой координатной п...,"Using the distance formula, we get that the di...","Используя формулу расстояния, получаем, что ра...",Algebra
297,Level 1,Simplify $16^{\frac{1}{2}}-625^{\frac{1}{2}}$.,\documentclass{article}\n\t\usepackage[utf8]{i...,We can simply plug in $16^{\frac{1}{2}}=4$ and...,Просто подставим $16^{\frac{1}{2}}=4$ и $625^{...,Algebra
298,Level 3,"Without using a calculator, compute $1003^2-99...",Вычислите $1003^2-997^2-1001^2+999^2$ без испо...,Applying the difference of squares factorizati...,Применяя факторизацию разности квадратов по от...,Algebra


Теперь я могу очень просто поделиться своим датасетом со всеми

In [78]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

In [80]:
from google.colab import userdata
hf_token = userdata.get('HuggingFaceToken')

In [82]:
dataset.push_to_hub("NMashalov/ru_tasks",token=hf_token)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/NMashalov/ru_tasks/commit/ec911bde953457ab5544bb578024add406d8abba', commit_message='Upload dataset', commit_description='', oid='ec911bde953457ab5544bb578024add406d8abba', pr_url=None, pr_revision=None, pr_num=None)

В следующем занятии.

Интерактивный поиск по учебнику, журналу